### Step 1: Read the data

Now that we have specified our file metadata, we can create a DataFrame. Notice that we use an *option* to specify that we want to infer the schema from the file. We can also explicitly set this to a particular schema if we have one already.

First, let's create a DataFrame in Python.

In [0]:
dbutils.widgets.removeAll()
dbutils.widgets.text("file_name", "")
dbutils.widgets.text("table_name", "")

In [0]:
file_name = getArgument("file_name")
file_type = "csv"
data_location = "/mnt/sparkcontainer"
table_name = getArgument("table_name")

### Step 2: Read the data

Now that we have specified our file metadata, we can create a DataFrame. Notice that we use an *option* to specify that we want to infer the schema from the file. We can also explicitly set this to a particular schema if we have one already.

First, let's create a DataFrame in Python.

In [0]:
source_location = "{0}/Triage/{1}.{2}".format(data_location, file_name, file_type)
df = spark.read.format(file_type).options(inferSchema='true', header='True').load(source_location)

display(df)

In [0]:
target_location = "{0}/Bronze/{1}".format(data_location, file_name)
df.write.format("delta").mode("overwrite").save(target_location)
#df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").partitionBy("Year", "Month").save(target_location)

In [0]:
spark.conf.set("tables.location", target_location)

In [0]:
%sql
SET table_Location = ${tables.location}

In [0]:
%sql
DROP TABLE IF EXISTS ${table_name};

CREATE TABLE ${table_name}
USING DELTA LOCATION '${tables.location}'

In [0]:
%sql
SELECT * FROM ${table_name}

In [0]:
%sql
DESCRIBE TABLE EXTENDED ${table_name}